In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import time
import numpy as np
np.set_printoptions(precision=3, linewidth=200, suppress=True)
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [3]:
from library.datasets.cifar10 import CIFAR10
from library.plot_tools import plot
from library.utils import file_utils

In [11]:
def freeze_graph(model_folder, model_name, output_node_name):
    print(model_folder)
    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path
    absolute_model_folder = '/'.join(input_checkpoint.split('/')[:-1])
    print(absolute_model_folder)
    output_graph = absolute_model_folder + '/' + model_name
    print(output_graph)
    output_node_names = output_node_name
    clear_devices = True
    print(input_checkpoint)
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)
        output_graph_def = graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            output_node_names.split(',')
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))
    return True

In [12]:
def load_graph(filename):
    with tf.gfile.GFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="prefix",
            op_dict=None,
            producer_op_list=None
        )
    return graph

In [13]:
dataset = 'cifar10'
file_no = 202
exp_no = 1
total_time = 0

In [14]:
model_folder = '../logs/' + dataset + '/' + str(file_no).zfill(3) + '_tfl_cnn/exp_no_' + str(exp_no).zfill(3) + '/'
model_name = 'residual_net_classifier.pb'
output_node_name = 'Predictions/predict_classes'

In [15]:
model_folder = './linear_model/'
model_name = 'linear_classifier_01.pb'
output_node_name = 'Predictions/predict_class'

In [16]:
device_name = '/cpu:0'
train_val_split_data = None
one_hot = True
num_images_required = 0.0
transform = True
transform_method = 'StandardScaler'

In [17]:
freeze_graph(model_folder, model_name, output_node_name)

./linear_model/
./linear_model
./linear_model/linear_classifier_01.pb
./linear_model/linear_classifier.ckpt-99
1
1
1
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.
11 ops in the final graph.


True

In [18]:
graph = load_graph(model_folder+'/'+model_name)

In [19]:
for op in graph.get_operations():
    print(op.name)

prefix/Inputs/Data/X_input
prefix/Parameters/Weights/Weight
prefix/Parameters/Weights/Weight/read
prefix/Parameters/Bias/Bias
prefix/Parameters/Bias/Bias/read
prefix/Predictions/mlp_layer/MatMul
prefix/Predictions/mlp_layer
prefix/Predictions/mlp_layer_1
prefix/Predictions/Softmax
prefix/Predictions/predict_class/dimension
prefix/Predictions/predict_class


In [20]:
x = graph.get_tensor_by_name('prefix/Inputs/Data/X_input:0')
y = graph.get_tensor_by_name('prefix/Predictions/predict_class:0')
print(str(x))
print(str(y))

Tensor("prefix/Inputs/Data/X_input:0", dtype=float32)
Tensor("prefix/Predictions/predict_class:0", shape=(?,), dtype=int64)


In [21]:
start = time.time()
one_hot = True
cifar10 = CIFAR10(one_hot_encode=one_hot, num_images=num_images_required, preprocess='StandardScaler',
                  train_validate_split=train_val_split_data, endian='little')
cifar10.load_data(train=False, test=True, data_directory='./datasets/cifar10/')
end = time.time()
print('[ Step 1] Loaded CIFAR 10 Dataset in %.4f ms' %((end-start)*1000))
total_time += (end-start)

Loading CIFAR 10 Dataset
MD5sum of the file: ./datasets/cifar10/cifar-10.tar.gz is verified
Loading 10000 test images
Loading CIFAR 10 Test Dataset
Unpickling test file: ./datasets/cifar10/cifar-10-batches/test_batch
Reading unpicked test file: ./datasets/cifar10/cifar-10-batches/test_batch


/net/voxel03/misc/me/praneethas/Softwares/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Loaded CIFAR 10 Dataset in 1.3864 seconds
[ Step 1] Loaded CIFAR 10 Dataset in 1386.9727 ms


In [22]:
sess = tf.Session(graph=graph)
prediction_numbers = sess.run(y, feed_dict={x: cifar10.test.data})

InternalError: Dst tensor is not initialized.
	 [[Node: _recv_prefix/Inputs/Data/X_input_0/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_18__recv_prefix/Inputs/Data/X_input_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

In [ ]:
prediction_classes = []
num_test_images = cifar10.test.data.shape[0]
for i in range(num_test_images):
    prediction_classes.append(cifar10.classes[int(prediction_numbers[i])])

In [ ]:
test_accuracy = accuracy_score(cifar10.test.class_labels, prediction_numbers)
print('Accuracy of the classifier on test dataset: %.4f' % test_accuracy)

In [ ]:
cifar10.plot_images(cifar10.test.data[:50], cifar10.test.class_names[:50], cls_pred=prediction_classes[:50], 
                    nrows=5, ncols=10, fig_size=(20,50), fontsize=30, convert=True)

In [ ]:
plot.plot_confusion_matrix(cifar10.test.class_labels, prediction_numbers, classes=cifar10.classes,
                           normalize=True, title='Confusion matrix')

In [ ]:
print('Detailed classification report')
print(classification_report(y_true=cifar10.test.class_labels, y_pred=prediction_numbers,
                            target_names=cifar10.classes))

In [ ]:
sess.close()